### Ingestion de la carpeta "production_company"

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-30')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo CSV usando DataFrameReader de Spark

In [0]:
from pyspark.sql.types import *

In [0]:
productions_companies_schema = StructType(fields=[
    StructField('companyId', IntegerType(), True),
    StructField('companyName', StringType(), True)
])

In [0]:
productions_companies_df = spark.read \
    .schema(productions_companies_schema) \
    .csv(f'{bronze_folder_path}/{v_file_date}/production_company')

### Paso 2 - Renombrar y Agregar Columnas
1. "companyId" renombrar a "company_id"
2. "companyName" renombrar a "company_name"
3. Agregar columna "ingestion_date"
4. Agregar columna "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_companies_final_df = add_ingestion_date(productions_companies_df) \
    .withColumnRenamed('companyId', 'company_id') \
    .withColumnRenamed('companyName', 'company_name') \
    .withColumn('enviroment', lit(v_enviroment)) \
    .withColumn('file_date', lit(v_file_date))

### Escribir la salida en formato "Parquet"

In [0]:
#Uso de UDF
#overwrite_partition(productions_companies_final_df, 'movie_silver', 'productions_companies', 'file_date')

In [0]:
merge_condition = 'tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(productions_companies_final_df, "movie_silver", "productions_companies", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.productions_companies
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f'{silver_folder_path}/productions_companies'))

In [0]:
dbutils.notebook.exit("Success")